# Pràctica 1 - Regressió

## Apartat (C): Analitzant Dades

Ens trobem dabant d'una base de dades respecte les **carcteristiques del formigó**, on trobem tant la seva **composició**, **edat de vida** i l'**esforç** que es capaç d'aguantar, sent aquest últim potser el que té més interés. Analitzarem més en profunditat aquestes dades per veure en profunditat les característiques de la base de dades.

In [154]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
%matplotlib notebook
from matplotlib import pyplot as plt
import scipy.stats

# Definim veure només els 2 primers decimals de les mostres
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Funció per llegir dades d'un csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',')
    return dataset

# Carregem la base de dades
dataset = load_dataset('Concrete_Data_Yeh.csv')
data = dataset.values

x = data[:, :2]
y = data[:, 2]

print("Forma de la base de dades:", dataset.shape)
print("Dimensions de la entrada x:", x.shape)
print("Dimensió dels atributs y:", y.shape)

Forma de la base de dades: (1030, 9)
Dimensions de la entrada x: (1030, 2)
Dimensió dels atributs y: (1030,)


Les etiquetes que ens indiquen les entrades son les que es poden veure a continuació:

In [155]:
print ('Etiquetes:', dataset.keys())

Etiquetes: Index(['cement', 'slag', 'flyash', 'water', 'superplasticizer',
       'coarseaggregate', 'fineaggregate', 'age', 'csMPa'],
      dtype='object')


Per començar a treballar amb aquest conjunt de dades, ens interessa saber quines variables poden tenir valors no existents. Podem observar que no hi ha cap valor nul en les mostres:

In [156]:
print(dataset.isnull().sum())

cement              0
slag                0
flyash              0
water               0
superplasticizer    0
coarseaggregate     0
fineaggregate       0
age                 0
csMPa               0
dtype: int64


A continuació, vuerem el tipus de dades que podem trobar dins de cada atribut.

In [157]:
dataset.head()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.00,0.00,0.00,162.00,2.50,1040.00,676.00,28,79.99
1,540.00,0.00,0.00,162.00,2.50,1055.00,676.00,28,61.89
2,332.50,142.50,0.00,228.00,0.00,932.00,594.00,270,40.27
3,332.50,142.50,0.00,228.00,0.00,932.00,594.00,365,41.05
4,198.60,132.40,0.00,192.00,0.00,978.40,825.50,360,44.30


In [158]:
dataset.describe()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
count,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00,1030.00
mean,281.17,73.90,54.19,181.57,6.20,972.92,773.58,45.66,35.82
std,104.51,86.28,64.00,21.35,5.97,77.75,80.18,63.17,16.71
min,102.00,0.00,0.00,121.80,0.00,801.00,594.00,1.00,2.33
25%,192.38,0.00,0.00,164.90,0.00,932.00,730.95,7.00,23.71
50%,272.90,22.00,0.00,185.00,6.40,968.00,779.50,28.00,34.45
75%,350.00,142.95,118.30,192.00,10.20,1029.40,824.00,56.00,46.14
max,540.00,359.40,200.10,247.00,32.20,1145.00,992.60,365.00,82.60


Una vegada ja ens hem començat a familiaritzar amb les dades, pasem a veure quines són les caracteristiques de les variables i la seva relació amb els altres atributs de la bade de dades.

In [159]:
import seaborn as sns
# Mirem la correlació entre els atributs d'entrada per entendre millor les dades
correlacio = dataset.corr()

plt.figure()

ax = sns.heatmap(correlacio, annot=True, linewidths=.5)

<IPython.core.display.Javascript object>

In [160]:
relacio = sns.pairplot(dataset)

<IPython.core.display.Javascript object>

Les conclusions que n'extraiem d'aquest petit anàlisin són les següents:

Cadascun atribut representa les característiques del formigó (intervals \[ \])
- cement: kg per cada $m^3$ de la barreja. La distrivució es troba principalment al primer i segon rang del interval. \[102, 504\] (double)
- blast furnace slag: kg per cada $m^3$ de la barreja. Trobem un volum important en el valor del 0, cosa que ens indica que hi ha moltes mostres que no conten aquest element. \[0, 359.4\] (double) 
- fly ash: kg per cada $m^3$ de la barreja. Trobem que més de la metitat de les mostres és 0. \[0, 210\] (double) 
- water: kg per cada $m^3$ de la barreja. Podem veure una distrivució semblan a una campana de Gauss, tot no cumplir-se del tot, tot i així, la seva concentració és bastant central. \[121.8, 247\] (double)
- superplasticizer: kg per cada $m^3$ de la barreja. Trobem un volum important en el valor del 0, cosa que ens indica que hi ha moltes mostres que no conten aquest element. \[0, 32.2\] (double) 
- coarse aggregate: kg per cada $m^3$ de la barreja. És una mostra bastant similar a una campana de Gauss. \[801, 1145\] (double)
- fine aggregate: kg per cada $m^3$ de la barreja.  És una mostra bastant similar a una campana de Gauss. \[594, 992.5\] (double)
- age: edat en dies. Els valors són bastant dispersos, no presentan cap caracteristiqua més enllà de que la majoria estan present en els 7 primers dies. \[1, 365\] (int)
- csMPa/compresive strength: força de compressió en MPa.  És una mostra bastant similar a una campana de Gauss. \[2.33, 82.6\] (double)

La força de compressió que és capaç d'aguantar el formigó és la que procederem a entrenar i predir en funció dels matrials que estan composats.

## Apartat (B): Primeres regressions 

Comencem inician una funció que ens dona l'error quadratic mitjà entre dos atributs.

In [161]:
import math

def mean_squeared_error(y1, y2):
    # comprovem que y1 i y2 tenen la mateixa mida
    assert(len(y1) == len(y2))
    mse = 0
    for i in range(len(y1)):
        mse += (y1[i] - y2[i])**2
    return mse / len(y1)

Important estandaritzar també els valors que tenim abans de procesar les dades per tenir una homogenieitat dins de les comparacions. Així afegim aquesta funció:

In [162]:
from sklearn.linear_model import LinearRegression

def regression(x, y):
    # Creem un objecte de regressió de sklearn
    regr = LinearRegression()

    # Entrenem el model per a predir y a partir de x
    regr.fit(x, y)

    # Retornem el model entrenat
    return regr

In [163]:
def standarize(x_train):
    mean = x_train.mean(0)
    std = x_train.std(0)
    x_t = x_train - mean[None, :]
    x_t /= std[None, :]
    return x_t

In [164]:
def split_data(x, y, train_ratio=0.8):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val

In [165]:
from sklearn.metrics import r2_score, mean_squared_error as mse

# Dividim dades d'entrenament
x = data[:, :-1]
y = data[:, -1]

x_train, y_train, x_val, y_val = split_data(x, y)

for i in range(x_train.shape[1]):
    x_t = x_train[:,i] # seleccionem atribut i en conjunt de train
    x_v = x_val[:,i] # seleccionem atribut i en conjunt de val.
    x_t = np.reshape(x_t,(x_t.shape[0],1))
    x_v = np.reshape(x_v,(x_v.shape[0],1))  

    regr = regression(x_t, y_train)    
    error = mse(y_val, regr.predict(x_v)) # calculem error
    r2 = r2_score(y_val, regr.predict(x_v))

    print("Error en atribut %d: %f" %(i, error))
    print("R2 score en atribut %d: %f" %(i, r2))

Error en atribut 0: 175.955257
R2 score en atribut 0: 0.329907
Error en atribut 1: 268.479079
R2 score en atribut 1: -0.022453
Error en atribut 2: 259.505661
R2 score en atribut 2: 0.011721
Error en atribut 3: 242.483022
R2 score en atribut 3: 0.076548
Error en atribut 4: 221.428470
R2 score en atribut 4: 0.156731
Error en atribut 5: 251.535979
R2 score en atribut 5: 0.042072
Error en atribut 6: 257.818336
R2 score en atribut 6: 0.018146
Error en atribut 7: 234.633818
R2 score en atribut 7: 0.106440


In [170]:
from sklearn.metrics import r2_score, mean_squared_error as mse

# Dividim dades d'entrenament
x = data[:, :-1]
y = data[:, -1]

x_train, y_train, x_val, y_val = split_data(x, y)

x_train_standarized = standarize(x_train)
x_val_standarized = standarize(x_val)

for i in range(x_train_standarized.shape[1]):
    x_t = x_train_standarized[:,i] # seleccionem atribut i en conjunt de train
    x_v = x_val_standarized[:,i] # seleccionem atribut i en conjunt de val.
    x_t = np.reshape(x_t,(x_t.shape[0],1))
    x_v = np.reshape(x_v,(x_v.shape[0],1))  

    regr = regression(x_t, y_train)    
    error = mse(y_val, regr.predict(x_v)) # calculem error
    r2 = r2_score(y_val, regr.predict(x_v))

    print("Error en atribut %d: %f" %(i, error))
    print("R2 score en atribut %d: %f" %(i, r2))

Error en atribut 0: 212.462708
R2 score en atribut 0: 0.298157
Error en atribut 1: 309.120711
R2 score en atribut 1: -0.021140
Error en atribut 2: 308.123746
R2 score en atribut 2: -0.017847
Error en atribut 3: 266.121100
R2 score en atribut 3: 0.120904
Error en atribut 4: 259.161918
R2 score en atribut 4: 0.143892
Error en atribut 5: 307.998678
R2 score en atribut 5: -0.017433
Error en atribut 6: 290.557276
R2 score en atribut 6: 0.040182
Error en atribut 7: 271.804081
R2 score en atribut 7: 0.102131


In [166]:
# WITH ALL VARIABLES: 

regr = regression(x_train, y_train)
error = mse(y_val, regr.predict(x_val))
r2 = r2_score(y_val, regr.predict(x_val))

print("WITH ALL VARIABLES")
print(f"MSE: {error}")
print(f"r2:  {r2}")

# MORE THAN 0.2 ABS CORRELATION

x_train1 = np.delete(x_train, [1, 2, 5, 6], axis=1)
x_val1 = np.delete(x_val, [1, 2, 5, 6], axis=1)

regr = regression(x_train1, y_train)
error = mse(y_val, regr.predict(x_val1))
r2 = r2_score(y_val, regr.predict(x_val1))

print("VARIABLES WITH MORE THAN 0.2 ABS CORRELATION")
print(f"MSE: {error}")
print(f"r2:  {r2}")

# MORE THAN 0.15 ABS CORRELATION

x_train2 = np.delete(x_train, [1, 2], axis=1)
x_val2 = np.delete(x_val, [1, 2], axis=1)

regr = regression(x_train2, y_train)
error = mse(y_val, regr.predict(x_val2))
r2 = r2_score(y_val, regr.predict(x_val2))

print("VARIABLES WITH MORE THAN 0.15 ABS CORRELATION")
print(f"MSE: {error}")
print(f"r2:  {r2}")

# NEGATIVE CORRELATION

x_train3 = np.delete(x_train, [2, 3, 5, 6], axis=1)
x_val3 = np.delete(x_val, [2, 3, 5, 6], axis=1)

regr = regression(x_train2, y_train)
error = mse(y_val, regr.predict(x_val2))
r2 = r2_score(y_val, regr.predict(x_val2))

print("VARIABLES NEGATIVE CORRELATION")
print(f"MSE: {error}")
print(f"r2:  {r2}")

WITH ALL VARIABLES
MSE: 94.2917399846862
r2:  0.6409073136373227
VARIABLES WITH MORE THAN 0.2 ABS CORRELATION
MSE: 116.40463657315796
r2:  0.5566944288130102
VARIABLES WITH MORE THAN 0.15 ABS CORRELATION
MSE: 99.94780129923225
r2:  0.6193672481766342
VARIABLES NEGATIVE CORRELATION
MSE: 99.94780129923225
r2:  0.6193672481766342


In [167]:
# WITH ALL VARIABLES: 

regr = regression(x_train, y_train)
error = mse(y_val, regr.predict(x_val))
r2 = r2_score(y_val, regr.predict(x_val))

print("WITH ALL VARIABLES")
print(f"MSE: {error}")
print(f"r2:  {r2}")

# WITHOUT I COLUMN

for i in range(x_train.shape[1]):

    x_train_i = np.delete(x_train, [i], axis=1)
    x_val_i = np.delete(x_val, [i], axis=1)
    
    regr = regression(x_train_i, y_train)
    error = mse(y_val, regr.predict(x_val_i))
    r2 = r2_score(y_val, regr.predict(x_val_i))

    print(f"WITHOUT {i}:")
    print(f"MSE: {error}")
    print(f"r2:  {r2}")


WITH ALL VARIABLES
MSE: 94.2917399846862
r2:  0.6409073136373227
WITHOUT 0:
MSE: 118.80159710129557
r2:  0.5475660470979837
WITHOUT 1:
MSE: 101.23993691576837
r2:  0.6144463881971471
WITHOUT 2:
MSE: 98.23899595089337
r2:  0.6258749179163758
WITHOUT 3:
MSE: 96.5571106980141
r2:  0.632280067441756
WITHOUT 4:
MSE: 95.49330210912713
r2:  0.6363313860834647
WITHOUT 5:
MSE: 93.96667653032074
r2:  0.6421452577996151
WITHOUT 6:
MSE: 94.29570943472434
r2:  0.6408921967195756
WITHOUT 7:
MSE: 135.69956378832532
r2:  0.48321326017626587


In [168]:
x_train_i = np.delete(x_train, [6, 5], axis=1)
x_val_i = np.delete(x_val, [6, 5], axis=1)
    
regr = regression(x_train_i, y_train)
error = mse(y_val, regr.predict(x_val_i))
r2 = r2_score(y_val, regr.predict(x_val_i))

print(f"WITHOUT 5 AND 6:")
print(f"MSE: {error}")
print(f"r2:  {r2}")

WITHOUT 5 AND 6:
MSE: 94.04644636473776
r2:  0.6418414691099996


In [169]:
# WITH ALL VARIABLES: 

x_train_standarized = standarize(x_train)
x_val_standarized = standarize(x_val)

regr = regression(x_train_standarized, y_train)
error = mse(y_val, regr.predict(x_val_standarized))
r2 = r2_score(y_val, regr.predict(x_val_standarized))

print("WITH ALL VARIABLES")
print(f"MSE: {error}")
print(f"r2:  {r2}")

# WITHOUT I COLUMN

for i in range(x_train_standarized.shape[1]):

    x_train_i_standarized = np.delete(x_train_standarized, [i], axis=1)
    x_val_i_standarized = np.delete(x_val_standarized, [i], axis=1)
    
    regr = regression(x_train_i_standarized, y_train)
    error = mse(y_val, regr.predict(x_val_i_standarized))
    r2 = r2_score(y_val, regr.predict(x_val_i_standarized))

    print(f"WITHOUT {i}:")
    print(f"MSE: {error}")
    print(f"r2:  {r2}")


WITH ALL VARIABLES
MSE: 91.31523970252145
r2:  0.6522427655279697
WITHOUT 0:
MSE: 119.0919338294125
r2:  0.5464603532623817
WITHOUT 1:
MSE: 101.34315546920952
r2:  0.6140532993895413
WITHOUT 2:
MSE: 96.62288362307744
r2:  0.6320295833977161
WITHOUT 3:
MSE: 92.79393060633406
r2:  0.6466114441732428
WITHOUT 4:
MSE: 92.79180592210672
r2:  0.6466195356409259
WITHOUT 5:
MSE: 91.50154262900405
r2:  0.6515332652222301
WITHOUT 6:
MSE: 91.74075186303214
r2:  0.6506222810102105
WITHOUT 7:
MSE: 133.33846763760548
r2:  0.4922050590301237
